## Notebook - Pat's work

In [5]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
from astroNN.datasets import load_galaxy10
import numpy as np
from sklearn.model_selection import train_test_split


In [6]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(f"Using device: {device}")

# Hyperparameters
num_epochs = 100
batch_size = 64
learning_rate = 0.001

Using device: cpu


In [8]:
def to_categorical(y, num_classes):
    return np.eye(num_classes)[y]

def load_images_labels():
    # To load images and labels (will download automatically at the first time)
    # First-time downloading location will be ~/.astroNN/datasets/
    images, labels = load_galaxy10()

    # To convert the labels to categorical 10 classes
    labels = to_categorical(labels, 10)

    # Combine labels and images loading and concatenation
    indices = [slice(10), slice(1000, 1010), slice(2000, 2010), slice(3000, 3010),
               slice(5000, 5010), slice(6000, 6010), slice(7000, 7010),
               slice(8000, 8010), slice(9000, 9010), slice(10000, 10010)]
    
    new_labels = np.concatenate([labels[idx] for idx in indices], axis=0)
    new_images = np.concatenate([images[idx] for idx in indices], axis=0)

    # Convert to desirable type
    new_labels = torch.tensor(new_labels[:100], dtype=torch.float32)
    new_images = torch.tensor(new_images[:100], dtype=torch.float32)

    images = torch.tensor(images, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32)
    
    #return new_images, new_labels
    return images, labels

def split_dataset(images, labels, test_size=0.1):
    train_idx, test_idx = train_test_split(np.arange(labels.shape[0]), test_size=test_size)
    train_images, train_labels, test_images, test_labels = images[train_idx], labels[train_idx], images[test_idx], labels[test_idx]
    return train_images, train_labels, test_images, test_labels


In [9]:
# Define your CNN model
class ComplexCNN(nn.Module):
    def __init__(self, num_classes):
        super(ComplexCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout_conv = nn.Dropout2d(0.25)
        self.dropout_fc = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.dropout_conv(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.dropout_conv(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        
        x = x.reshape(-1, 128 * 32 * 32)
        x = self.dropout_fc(x)
        x = self.relu(self.fc1(x))
        x = self.dropout_fc(x)
        x = self.fc2(x)
        
        return x

In [10]:
# Load your data and move to GPU
images, labels = load_images_labels()
images = images.permute(0, 3, 1, 2).to(device)
labels = labels.to(device)

C:\Users\dvirz\.astroNN\datasets\Galaxy10_DECals.h5 was found!


In [ ]:
# Normalize the images
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
images = normalize(images)

# Split the data into training and testing sets
train_images, train_labels, test_images, test_labels = split_dataset(images, labels)

# Create DataLoader for training set
train_dataset = list(zip(train_images, train_labels))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create DataLoader for testing set
test_dataset = list(zip(test_images, test_labels))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Instantiate the model and move to GPU
num_classes = 10
model = ComplexCNN(num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)  # Move to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.argmax(dim=1).type(torch.long))  # Ensure labels are torch.long
        loss.backward()

        # Optional: Gradient Clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")

# Save the trained model if needed
# torch.save(model.state_dict(), 'trained_model.pth')

In [ ]:
# Testing loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch[0], batch[1]
        outputs = model(inputs)
        predicted = outputs.argmax(1)
        total += labels.size(0)
        labels = labels.argmax(1)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")